# 📊 IORAG Experiment Notebook

> **Data Management Guide**: 本notebook使用标准化的路径配置。请阅读 [README_DATA.md](../../README_DATA.md) 了解数据管理最佳实践。

## 📁 目录结构
```
DarshanRAG/
├── data/                    # 数据文件（不提交git）
│   ├── raw/                 # 原始日志
│   ├── parsed/              # 解析后的日志
│   ├── archives/            # 归档文件
│   └── examples/            # 示例文件
├── knowledge_graphs/        # KG文件（不提交git）
├── experiments/
│   ├── notebooks/           # 📍 当前位置
│   ├── scripts/             # 实验脚本
│   ├── results/             # 结果输出（不提交git）
│   └── storage/             # RAG存储（不提交git）
```

## ⚠️ 重要提示
1. **不要硬编码路径** - 使用 `config_paths` 模块
2. **API Key** - 使用环境变量，不要硬编码在代码中
3. **大文件** - 所有大文件自动被git忽略，不会提交

## 📝 Git提交时推荐的完整流程
```
# Step 1: 检查状态
git status

# Step 2: 添加所有更改（.gitignore会自动过滤大文件）
git add -A

# Step 3: 检查暂存区（确保没有大文件）
git status
git diff --cached --stat

# Step 4: 如果一切正常，提交
git commit -m "Finish KG building"

# Step 5: 推送
git push origin main
```

## 🚀 Setup

In [1]:
# ============================================================
# API Key 配置 - 使用环境变量（推荐）
# ============================================================
import os

# ⚠️ 不要硬编码API Key！使用环境变量
# 方式1: 从环境变量读取（推荐）
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️  OPENAI_API_KEY 环境变量未设置")
    print("请在终端中运行: export OPENAI_API_KEY='your-key-here'")
    # 或者临时设置（仅用于测试，不要提交到git）
    # os.environ["OPENAI_API_KEY"] = "your-key-here"
else:
    print("✅ OPENAI_API_KEY 已加载")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

✅ OPENAI_API_KEY 已加载


In [2]:
# ============================================================
# 路径配置 - 使用 config_paths 模块
# ============================================================
import sys
from pathlib import Path

# 添加experiments目录到Python路径
project_root = Path('/users/Minqiu/DarshanRAG')
sys.path.insert(0, str(project_root / 'experiments'))

from config_paths import PATHS, get_parsed_log_path, get_kg_path, get_storage_dir

# 配置实验数据路径
EXPERIMENT_DATE = '2025-1-1'
PARSED_LOGS = get_parsed_log_path(EXPERIMENT_DATE)
KG_FILE = get_kg_path('kg', EXPERIMENT_DATE)
STORAGE_DIR = get_storage_dir(f'lightrag_{EXPERIMENT_DATE.replace("-", "_")}')

# 确保结果输出目录存在
RESULTS_DIR = PATHS['results'] / f'iorag_{EXPERIMENT_DATE}'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("✅ 路径配置完成:")
print(f"  📂 Parsed logs: {PARSED_LOGS}")
print(f"  🕸️  KG file: {KG_FILE}")
print(f"  💾 Storage: {STORAGE_DIR}")
print(f"  📊 Results: {RESULTS_DIR}")

✅ All directories ensured under: /users/Minqiu/DarshanRAG
✅ 路径配置完成:
  📂 Parsed logs: /users/Minqiu/DarshanRAG/data/parsed/parsed-logs-2025-1-1
  🕸️  KG file: /users/Minqiu/DarshanRAG/knowledge_graphs/kg_2025-1-1.json
  💾 Storage: /users/Minqiu/DarshanRAG/experiments/storage/lightrag_2025_1_1
  📊 Results: /users/Minqiu/DarshanRAG/experiments/results/iorag_2025-1-1


In [ ]:
%uv pip install tqdm, torch, transformers

# **Parse log**

In [6]:
!chmod +x /users/Minqiu/DarshanRAG/experiments/scripts/unpack-darshan-logs.sh
!/users/Minqiu/DarshanRAG/experiments/scripts/unpack-darshan-logs.sh /users/Minqiu/polaris-darshan-logs-25-1

Extracting: /users/Minqiu/polaris-darshan-logs-25-1/8/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/21/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/24/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/2/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/10/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/11/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/26/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/15/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/20/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/30/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/19/logs.tar.gz
Done: /users/Minqiu/polaris-darshan-logs-25-1/8/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/1/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/13/logs.tar.gz
Extracting: /users/Minqiu/polaris-darshan-logs-25-1/27/logs.tar.gz
Extra

In [8]:
# 假设 parent_dir 是你想要遍历的父文件夹路径
parent_dir = '/users/Minqiu/polaris-darshan-logs-25-1'  # 修改为你的实际父文件夹

import os
import subprocess
from pathlib import Path

parsed_root = Path('~/parsed-logs-25-1').expanduser()
parsed_root.mkdir(parents=True, exist_ok=True)

darshan_logs = []
for root, dirs, files in os.walk(parent_dir):
    for file in files:
        if file.endswith('.darshan') or '.darshan.' in file:
            fullpath = os.path.join(root, file)
            # 生成相对于parent_dir的路径
            rel = os.path.relpath(fullpath, parent_dir)
            output_dir = parsed_root / Path(rel).parent
            output_dir.mkdir(parents=True, exist_ok=True)
            out_txt = output_dir / (Path(file).stem + '.txt')
            darshan_logs.append((fullpath, out_txt))

from tqdm import tqdm

# 统计成功和失败的数量
success_count = 0
fail_count = 0

for in_log, out_txt in tqdm(darshan_logs, desc="Parsing darshan logs"):
    # 使用 subprocess 运行命令并检查返回码
    result = subprocess.run(
        ['darshan-parser', in_log],
        capture_output=True,
        text=True
    )
    
    # 只在命令成功时写入文件
    if result.returncode == 0:
        with open(out_txt, 'w') as f:
            f.write(result.stdout)
        success_count += 1
    else:
        # 命令失败，不创建空文件
        fail_count += 1
        # 可选：记录失败的日志
        # print(f"Failed to parse {in_log}: {result.stderr.strip()}")

print(f"\n✅ 解析完成: 成功 {success_count} 个, 失败 {fail_count} 个")

Parsing darshan logs: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24908/24908 [03:39<00:00, 113.59it/s]


✅ 解析完成: 成功 24392 个, 失败 516 个


可以加上失败原因

# **Process log (creating original_name_signals_v2.2.txt)**

In [9]:
# 1. 处理单个文件
!python3 /users/Minqiu/DarshanRAG/experiments/scripts/process_darshan_signals_v2.4.py \
    /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example.txt \
    -o /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.4.txt

Processing: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example.txt
  -> Output: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.4.txt

Processing complete!


In [ ]:
# 2. 处理文件夹
!python3 /users/Minqiu/DarshanRAG/experiments/scripts/process_darshan_signals_v2.4.py \
    /path/to/logs/ \
    -o ./my_signals/

# **Building KG (creating original_name_kg_v1.txt)**

In [ ]:
!python /users/Minqiu/DarshanRAG/experiments/scripts/darshan_kg_builder_v2.1.py \
    -i /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.4.txt \
    -o /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_v2.1.json

Parsing single file: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_signals_v2.4.txt

Building Knowledge Graph...

Knowledge Graph Statistics:
Total entities: 68
Total relationships: 99

Entity breakdown:
  APPLICATION: 1
  FILE: 22
  FILESYSTEM: 1
  JOB: 1
  MODULE: 3
  RECORD: 40

Output saved to: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_v2.1.json


# **Extracting Summary and updating KG**

In [11]:
!python3 /users/Minqiu/DarshanRAG/experiments/scripts/generate_descriptions_v3.py \
    /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_v2.1.json \
    /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_with_desc_v3.json


Reading /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_v2.1.json...
Processing 68 entities and 99 relationships...
Writing to /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_with_desc_v3.json...
Done!

使用统计报告

【1】实体模板中永远没有匹配到的属性（总是 N/A）:
----------------------------------------------------------------------

  APPLICATION:
    - nprocs (缺失1次)

  FILE:
    - rank (缺失22次)
    - is_shared (缺失22次)

  JOB:
    - nnodes (缺失1次)
    - exe (缺失1次)

  RECORD:
    - io_start_ts (缺失40次)
    - seq_write_ratio (缺失40次)
    - consec_write_ratio (缺失40次)

【2】关系模板中永远没有匹配到的属性（总是 N/A）:
----------------------------------------------------------------------

  MODULE→RECORD:
    - operation_types (缺失40次)


【3】实体 JSON 中永远没有用到的属性:
----------------------------------------------------------------------

  FILE (1个未使用属性):
    - file_path_norm

  JOB (6个未使用属性):
    - job_io_summary
    - mount_table
    - total_bytes_read
    - total_bytes_written
    - total_reads
    - total_writes

 

# **Parse & Add Chunks**

In [12]:
!python3 /users/Minqiu/DarshanRAG/experiments/scripts/parse_darshan_chunks.py \
  --log /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example.txt \
  --kg /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_with_desc_v3.json \
  --output /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_with_chunks_v3.json

Searching for log files in: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example.txt
✓ Found 1 log file(s)

Processing: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example.txt
✓ Parsed 3957 counter lines
✓ Grouped into 69 entities

Adding chunks to KG...
✓ Updated KG with 68 chunks
✓ Saved to: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_with_chunks_v3.json
✓ Statistics: 66/68 entities have counter data
✓ Done!


# **Embedding KG**

In [18]:
# 直接用这个命令！
!python3 /users/Minqiu/DarshanRAG/experiments/scripts/embed_kg_local.py \
  --kg /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_with_chunks.json \
  --output /users/Minqiu/DarshanRAG/data/examples/embeddings_gemma \
  --model google/embeddinggemma-300m \
  --batch-size 4 \
  --max-length 2048


Loading KG from: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_with_chunks.json
✓ Loaded KG:
  - 68 entities
  - 99 relationships

Loading embedding model: google/embeddinggemma-300m
config.json: 100%|█████████████████████████| 1.49k/1.49k [00:00<00:00, 11.2MB/s]
tokenizer_config.json: 100%|███████████████| 1.16M/1.16M [00:00<00:00, 9.79MB/s]
tokenizer.json: 100%|██████████████████████| 33.4M/33.4M [00:00<00:00, 40.9MB/s]
added_tokens.json: 100%|██████████████████████| 35.0/35.0 [00:00<00:00, 304kB/s]
special_tokens_map.json: 100%|█████████████████| 662/662 [00:00<00:00, 3.19MB/s]
model.safetensors: 100%|████████████████████| 1.21G/1.21G [00:02<00:00, 417MB/s]
Loading weights: 100%|█| 314/314 [00:00<00:00, 3129.40it/s, Materializing param=
✓ Model loaded on device: cpu
✓ Model embedding dimension: 768

Preparing entity texts...
✓ Prepared 68 entity texts

Embedding entities...
Embedding: 100%|████████████████████████████████| 17/17 [00:07<00:00,  2.22it/s]
✓ Generated e

# **IORAG!**

In [21]:
!python3 /users/Minqiu/DarshanRAG/experiments/scripts/load_custom_kg_to_lightrag.py \
  --kg /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_with_chunks.json \
  --embeddings /users/Minqiu/DarshanRAG/data/examples/embeddings_gemma \
  --embedding-model google/embeddinggemma-300m


📋 Configuration:
  KG: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_with_chunks.json
  Working Dir: ./lightrag_darshan_storage
  Pre-computed Embeddings: /users/Minqiu/DarshanRAG/data/examples/embeddings_gemma
  Embedding Model: google/embeddinggemma-300m
  OpenAI Model: gpt-4o-mini
  OpenAI Key: sk-proj-...9eQA

🚀 Initializing LightRAG with Custom KG

📂 Loading KG from: /users/Minqiu/DarshanRAG/data/examples/Darshan_log_example_kg_with_chunks.json
✓ Loaded KG:
  - Entities: 68
  - Relationships: 99
  - Chunks: 68

💾 Loading pre-computed embeddings from: /users/Minqiu/DarshanRAG/data/examples/embeddings_gemma
✓ Found pre-computed embeddings!
  - Will use pre-computed embeddings for KG entities and relationships
  - Embedding model will only be used for new queries

🔧 Creating local embedding function...
📦 Loading embedding model: google/embeddinggemma-300m
Loading weights: 100%|█| 314/314 [00:00<00:00, 3130.92it/s, Materializing param=
✓ Model loaded on cpu, dimension

# **Visualize IORAG**

*3. 浏览器打开 http://localhost:9621*

*就可以用Web UI查询和可视化图谱了*

# **Query Testing**

1.naive: 简单向量搜索

    result = await query("question", mode="naive")

2.local: 基于实体的局部搜索（适合具体细节）

    result = await query("question", mode="local")

3.global: 基于关系的全局搜索（适合整体概况）

    result = await query("question", mode="global")

4.hybrid: 结合 local + global（推荐，平衡）

    result = await query("question", mode="hybrid")

5.mix: 混合图谱和向量检索
    
    result = await query("question", mode="mix")


In [23]:
import os
import asyncio
from lightrag import LightRAG, QueryParam
from lightrag.llm.openai import openai_complete_if_cache
from lightrag.utils import EmbeddingFunc
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from typing import List

print("✓ Imports successful!")


/users/Minqiu/DarshanRAG/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Imports successful!


In [24]:
class LocalEmbeddingFunction:
    def __init__(self, model_name="google/embeddinggemma-300m", device=None, batch_size=4):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(self.device).eval()
        self.embedding_dim = self.model.config.hidden_size
        self.batch_size = batch_size
        print(f"✓ Model loaded: {model_name} ({self.embedding_dim}D)")

    async def __call__(self, texts):
        embeddings = []
        with torch.no_grad():
            for i in range(0, len(texts), self.batch_size):
                batch = texts[i:i + self.batch_size]
                encoded = self.tokenizer(batch, padding=True, truncation=True,
                                        max_length=512, return_tensors="pt").to(self.device)
                outputs = self.model(**encoded)
                attention_mask = encoded['attention_mask']
                token_embeddings = outputs.last_hidden_state
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
                batch_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / \
                                  torch.clamp(input_mask_expanded.sum(1), min=1e-9)
                batch_embeddings = torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
                embeddings.append(batch_embeddings.cpu().numpy())
        return np.vstack(embeddings)

# 创建 embedding 函数
local_embed = LocalEmbeddingFunction()
embedding_func = EmbeddingFunc(
    embedding_dim=local_embed.embedding_dim,
    max_token_size=2048,
    func=lambda texts: local_embed(texts)
)


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 314/314 [00:00<00:00, 954.96it/s, Materializing param=norm.weight]


✓ Model loaded: google/embeddinggemma-300m (768D)


In [25]:
async def llm_model_func(prompt, system_prompt=None, history_messages=[], **kwargs):
    return await openai_complete_if_cache(
        "gpt-4o-mini",  # 或 "gpt-4o" 获得更好质量
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        api_key=os.environ['OPENAI_API_KEY'],
        **kwargs
    )

print("✓ LLM function configured")


✓ LLM function configured


In [26]:
# 加载你之前用脚本创建的 LightRAG 实例
WORKING_DIR = "./lightrag_darshan_storage"  # ← 你的 working directory

rag = LightRAG(
    working_dir=WORKING_DIR,
    embedding_func=embedding_func,
    llm_model_func=llm_model_func,
)

await rag.initialize_storages()
print("✓ LightRAG loaded successfully!")
print(f"  Working directory: {WORKING_DIR}")

INFO: [] Loaded graph from ./lightrag_darshan_storage/graph_chunk_entity_relation.graphml with 862 nodes, 698 edges
INFO:nano-vectordb:Load (862, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './lightrag_darshan_storage/vdb_entities.json'} 862 data
INFO:nano-vectordb:Load (698, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './lightrag_darshan_storage/vdb_relationships.json'} 698 data
INFO:nano-vectordb:Load (107, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './lightrag_darshan_storage/vdb_chunks.json'} 107 data
INFO: [] Process 4290 KV load full_docs with 20 records
INFO: [] Process 4290 KV load text_chunks with 107 records
INFO: [] Process 4290 KV load full_entities with 20 records
INFO: [] Process 4290 KV load full_relations with 20 records
INFO: [] Process 4290 KV load entity_chunks with 862 records
INFO: [] Process 4290 KV load relation_chunk

✓ LightRAG loaded successfully!
  Working directory: ./lightrag_darshan_storage


In [27]:
async def query(question, mode="hybrid", top_k=5):
    """查询你的 Darshan KG"""
    result = await rag.aquery(
        question,
        param=QueryParam(mode=mode, top_k=top_k)
    )
    return result

print("✓ Query helper ready!")


✓ Query helper ready!


In [28]:
# Cell: 基本查询
result = await query("What are the POSIX I/O operations?")
print(result)


INFO: LLM func: 4 new workers initialized (Timeouts: Func: 180s, Worker: 360s, Health Check: 375s)
INFO:  == LLM cache == saving: hybrid:keywords:67ffa595c71ecf142e23f7e91a1776dd
INFO: Embedding func: 8 new workers initialized (Timeouts: Func: 30s, Worker: 60s, Health Check: 75s)
INFO: Query nodes: Read, Write, Open, Close, Seek, File descriptors (top_k:5, cosine:0.2)
INFO: Local query: 5 entites, 3 relations
INFO: Query edges: POSIX I/O operations, File handling, Input/Output standards (top_k:5, cosine:0.2)
INFO: Global query: 7 entites, 5 relations
INFO: Raw search results: 12 entities, 8 relations, 0 vector chunks
INFO: After truncation: 12 entities, 8 relations
INFO: Selecting 30 from 73 entity-related chunks by vector similarity
INFO: Find 4 additional chunks in 3 relations (deduplicated 3)
INFO: Selecting 4 from 4 relation-related chunks by vector similarity
INFO: Round-robin merged chunks: 34 -> 34 (deduplicated 0)
INFO: Final context: 12 entities, 8 relations, 20 chunks
INFO: F

POSIX (Portable Operating System Interface) defines a set of standards that govern how file operations are performed across different systems. These operations encompass a variety of essential tasks related to file handling. The key POSIX I/O operations include:

- **Open:** This operation involves creating or opening a file for reading, writing, or both. It initializes a file descriptor that allows access to the file.

- **Close:** After file processing is completed, this operation releases the file descriptor, effectively closing the file and ensuring any buffered data is written.

- **Read:** This operation reads data from a file into a buffer. Based on the POSIX metrics, the total bytes read during operations can be measured, providing insights into data consumption.

- **Write:** This operation is used to write data from a buffer to a file. It can also be tracked to monitor how much data has been successfully written.

- **Seek:** This function allows changing the current position

In [29]:
# Cell: 查询文件访问
result = await query("Which files were accessed and where?")
print(result)


INFO:  == LLM cache == saving: hybrid:keywords:4dd7b1e1c3c9114f06df7bac7556e02f
INFO: Query nodes: Access logs, File names, File locations, User activity (top_k:5, cosine:0.2)
INFO: Local query: 5 entites, 13 relations
INFO: Query edges: File access, File tracking, Data retrieval (top_k:5, cosine:0.2)
INFO: Global query: 9 entites, 5 relations
INFO: Raw search results: 11 entities, 15 relations, 0 vector chunks
INFO: After truncation: 11 entities, 15 relations
INFO: Selecting 27 from 64 entity-related chunks by vector similarity
INFO: Find 2 additional chunks in 2 relations (deduplicated 5)
INFO: Selecting 2 from 2 relation-related chunks by vector similarity
INFO: Round-robin merged chunks: 29 -> 29 (deduplicated 0)
INFO: Final context: 11 entities, 15 relations, 20 chunks
INFO: Final chunks S+F/O: E1/1 R1/1 E3/2 R1/2 E1/3 E3/4 E2/5 E2/6 E1/7 E1/8 E2/9 E1/10 E3/11 E1/12 E1/13 E2/14 E3/15 E2/16 E3/17 E2/18
INFO:  == LLM cache == saving: hybrid:query:eb2e102715159d0ef7623a1ca5a89f78


The provided information does not explicitly list the specific files that were accessed. However, multiple entries refer to file operations and metrics analyzed under POSIX standards, such as access counts and timestamps related to operations performed within specific home directories.

Here are some common aspects noted about file accesses:

1. **Access Counts**: 
   - Numerous entries reference access metrics, often recorded for specific paths such as `/home/727026018`, indicating how many times files were accessed using various operations (e.g., reads and writes).

2. **File Operations**:
   - Entries record operations like opens, reads, and writes, with associated metrics gathered in directories like:
     - `/home/727026018` where 4 reads were noted.
     - `/home/392142400` and `/home/365049253` also indicate access activities, although specific files are not detailed.

3. **Timestamps**:
   - Metrics for operations are accompanied by timestamp data, noting when kinds of operatio

In [31]:
# Cell: 自定义查询
result = await rag.aquery(
    "Analyze the I/O performance",
    param=QueryParam(
        mode="hybrid",
        top_k=10,
        only_need_context=False
    )
)
print(result)


INFO:  == LLM cache == saving: hybrid:keywords:eabc85a266260910765c034dab59e3de
INFO: Query nodes: Input/Output, Throughput, Latency, Disk speed, Benchmarking (top_k:10, cosine:0.2)
INFO: Local query: 10 entites, 13 relations
INFO: Query edges: I/O performance, Performance analysis (top_k:10, cosine:0.2)
INFO: Global query: 15 entites, 10 relations
INFO: Raw search results: 22 entities, 21 relations, 0 vector chunks
INFO: After truncation: 22 entities, 21 relations
INFO: Selecting 55 from 74 entity-related chunks by vector similarity
INFO: Find 2 additional chunks in 2 relations (deduplicated 14)
INFO: Selecting 2 from 2 relation-related chunks by vector similarity
INFO: Round-robin merged chunks: 57 -> 57 (deduplicated 0)
INFO: Final context: 22 entities, 21 relations, 20 chunks
INFO: Final chunks S+F/O: E7/1 R2/1 E6/2 R1/2 E2/3 E4/4 E2/5 E5/6 E8/7 E3/8 E4/9 E1/10 E2/11 E5/12 E8/13 E2/14 E8/15 E9/16 E8/17 E1/18
INFO:  == LLM cache == saving: hybrid:query:87c87adf1c6284a4514e63354ea22e

To analyze the I/O performance, we can consider various performance metrics and statistics derived from the data provided, particularly focusing on measurements in the context of Standard Input/Output (STDIO) and POSIX standards.

### Key Performance Metrics
1. **Read/Write Operations**:
   - **STDIO Opens**: The number of files opened via STDIO.
   - **STDIO Reads/Writes**: The count of read and write operations executed. For instance, the number of writes recorded ranges from 0 to several hundred, depending on the specific operation context.
   - **STDIO Bytes Read/Written**: This metric identifies the total byte count for read and write operations. Values can range significantly based on operation requirements, showing the data volume handled.

2. **Maximum Bytes**:
   - **STDIO_MAX_BYTE_READ** and **STDIO_MAX_BYTE_WRITTEN** indicate the upper limits for bytes that can be processed in a single I/O operation. These metrics are crucial for understanding the efficiency and effectivenes